# Домашнее задание 21: Предобработка текста на корпусе Helsinki-NLP/opus-100

In [23]:
# !pip install datasets nltk pymorphy3 natasha

## Импорт библиотек и подготовка ресурсов
Загружаем инструменты и необходимые языковые модели.

In [24]:
from datasets import load_dataset
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import pymorphy3
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab, Doc
from huggingface_hub import hf_hub_download

nltk.download('punkt')
nltk.download('stopwords')
morph = pymorphy3.MorphAnalyzer()
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\stepa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\stepa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Загрузка и первичный анализ корпуса

Ограничим объём для отладки и анализа до 200 предложений

In [25]:
pair = "en-ru"
LANG_SRC, LANG_TGT = pair.split("-")

DATASET_REPO = "Helsinki-NLP/opus-100"

def load_split(split: str):
    filename = f"{LANG_SRC}-{LANG_TGT}/{split}-00000-of-00001.parquet"
    local_path = hf_hub_download(DATASET_REPO, filename=filename, repo_type="dataset")
    df = pd.read_parquet(local_path)
    return [{"translation": rec} for rec in df["translation"].tolist()]

train_ds = load_split("train")
val_ds   = load_split("validation")
test_ds  = load_split("test")

print(f"Размеры исходных сплитов: train={len(train_ds)}, val={len(val_ds)}, test={len(test_ds)}")

Размеры исходных сплитов: train=1000000, val=2000, test=2000


In [26]:
sample_size = 200  # ограничим объём для отладки и анализа
train_subset = train_ds[:sample_size]
df = pd.DataFrame({
    'ru': [row['translation'][LANG_TGT] for row in train_subset],
    'en': [row['translation'][LANG_SRC] for row in train_subset],
})
df.head()

,ru,en
0,"Да, но не совсем...","Yeah, that's not exactly..."
1,!,!
2,Приводимое ниже расписание является предварите...,The schedule below is tentative; up-to-date in...
3,"Но сейчас ...я вверяю вам удостовериться, что ...","But for now,"
4,Они тусовались там несколько недель.,He'd been out there a few weeks or so.


## Токенизация предложений с помощью NLTK
Разбиваем русские предложения на слова.

In [27]:
df['tokens'] = df['ru'].apply(word_tokenize)
df[['ru', 'tokens']].head()

,ru,tokens
0,"Да, но не совсем...","[Да, ,, но, не, совсем, ...]"
1,!,[!]
2,Приводимое ниже расписание является предварите...,"[Приводимое, ниже, расписание, является, предв..."
3,"Но сейчас ...я вверяю вам удостовериться, что ...","[Но, сейчас, ..., я, вверяю, вам, удостоверить..."
4,Они тусовались там несколько недель.,"[Они, тусовались, там, несколько, недель, .]"


## Приведение токенов к нижнему регистру
Упрощаем сравнение слов.

In [28]:
df['tokens_lower'] = df['tokens'].apply(lambda tokens: [token.lower() for token in tokens])
df[['tokens', 'tokens_lower']].head()

,tokens,tokens_lower
0,"[Да, ,, но, не, совсем, ...]","[да, ,, но, не, совсем, ...]"
1,[!],[!]
2,"[Приводимое, ниже, расписание, является, предв...","[приводимое, ниже, расписание, является, предв..."
3,"[Но, сейчас, ..., я, вверяю, вам, удостоверить...","[но, сейчас, ..., я, вверяю, вам, удостоверить..."
4,"[Они, тусовались, там, несколько, недель, .]","[они, тусовались, там, несколько, недель, .]"


## Лемматизация токенов с помощью pymorphy3
Получаем нормальные формы слов.

In [29]:
def lemmatize_tokens(tokens):
    lemmas = []
    for token in tokens:
        if re.search('[а-яА-ЯёЁ]', token):
            parsed = morph.parse(token)[0]
            lemmas.append(parsed.normal_form)
    return lemmas
df['lemmas'] = df['tokens_lower'].apply(lemmatize_tokens)
df[['tokens_lower', 'lemmas']].head()

,tokens_lower,lemmas
0,"[да, ,, но, не, совсем, ...]","[да, но, не, совсем]"
1,[!],[]
2,"[приводимое, ниже, расписание, является, предв...","[приводить, ниже, расписание, являться, предва..."
3,"[но, сейчас, ..., я, вверяю, вам, удостоверить...","[но, сейчас, я, вверять, вы, удостовериться, ч..."
4,"[они, тусовались, там, несколько, недель, .]","[они, тусоваться, там, несколько, неделя]"


## Очистка от стоп-слов и знаков препинания
Удаляем часто встречающиеся и неинформативные слова.

In [30]:
russian_stopwords = set(stopwords.words('russian'))
def clean_tokens(tokens):
    cleaned = []
    for token in tokens:
        if re.fullmatch(r"[а-яё-]+", token) and token not in russian_stopwords:
            cleaned.append(token)
    return cleaned
df['clean_tokens'] = df['lemmas'].apply(clean_tokens)
df[['lemmas', 'clean_tokens']].head()

,lemmas,clean_tokens
0,"[да, но, не, совсем]",[]
1,[],[]
2,"[приводить, ниже, расписание, являться, предва...","[приводить, ниже, расписание, являться, предва..."
3,"[но, сейчас, я, вверять, вы, удостовериться, ч...","[вверять, удостовериться, шотландец, приуменьш..."
4,"[они, тусоваться, там, несколько, неделя]","[тусоваться, несколько, неделя]"


## Морфологический анализ одного предложения
Сравниваем результаты pymorphy3 и Natasha для выбранного примера.

In [31]:
target_column = LANG_TGT
example_sentence = df[target_column].dropna().iloc[2]
example_sentence

'Приводимое ниже расписание является предварительным; с самой последней информацией можно ознакомиться в Интернете по адресу www.un.org/News/ossg/conf.htm.'

In [32]:
print(f'Пример предложения ({target_column}):')
print(example_sentence)
print()
print('pymorphy3:')
for token in word_tokenize(example_sentence):
    if re.search('[а-яА-ЯёЁ]', token):
        parsed = morph.parse(token)[0]
        print(f'{token:>15} -> {parsed.normal_form:>15} | {parsed.tag}')
print()

Пример предложения (ru):
Приводимое ниже расписание является предварительным; с самой последней информацией можно ознакомиться в Интернете по адресу www.un.org/News/ossg/conf.htm.

pymorphy3:
     Приводимое ->       приводить | PRTF,impf,tran,pres,pssv neut,sing,nomn
           ниже ->            ниже | PREP
     расписание ->      расписание | NOUN,inan,neut sing,nomn
       является ->        являться | VERB,impf,intr sing,3per,pres,indc
предварительным -> предварительный | ADJF plur,datv
              с ->               с | PREP
          самой ->             сам | ADJF,Apro femn,sing,gent
      последней ->       последний | ADJF femn,sing,gent
    информацией ->      информация | NOUN,inan,femn sing,ablt
          можно ->           можно | PRED,pres
   ознакомиться ->    ознакомиться | INFN,perf,intr
              в ->               в | PREP
      Интернете ->        интернет | NOUN,inan,masc sing,loct
             по ->              по | PREP
         адресу ->           адрес 

In [33]:
print('Natasha:')
doc = Doc(example_sentence)
doc.segment(segmenter)
doc.tag_morph(morph_tagger)
for token in doc.tokens:
    if token.pos:
        token.lemmatize(morph_vocab)
        print(f'{token.text:>15} -> {token.lemma:>15} | {token.pos} | {token.feats}')

Natasha:
     Приводимое ->       приводить | PROPN | {'Animacy': 'Anim', 'Case': 'Gen', 'Gender': 'Masc', 'Number': 'Sing'}
           ниже ->            ниже | ADJ | {'Degree': 'Cmp'}
     расписание ->      расписание | NOUN | {'Animacy': 'Inan', 'Case': 'Nom', 'Gender': 'Neut', 'Number': 'Sing'}
       является ->        являться | VERB | {'Aspect': 'Imp', 'Mood': 'Ind', 'Number': 'Sing', 'Person': '3', 'Tense': 'Pres', 'VerbForm': 'Fin', 'Voice': 'Mid'}
предварительным -> предварительный | ADJ | {'Case': 'Ins', 'Degree': 'Pos', 'Gender': 'Neut', 'Number': 'Sing'}
              ; ->               ; | PUNCT | {}
              с ->               с | ADP | {}
          самой ->             сам | ADJ | {'Case': 'Ins', 'Degree': 'Pos', 'Gender': 'Fem', 'Number': 'Sing'}
      последней ->       последний | ADJ | {'Case': 'Ins', 'Degree': 'Pos', 'Gender': 'Fem', 'Number': 'Sing'}
    информацией ->      информация | NOUN | {'Animacy': 'Inan', 'Case': 'Ins', 'Gender': 'Fem', 'Number': 'Si

## Итоговая таблица с результатами обработки
Сводим ключевые шаги по первым пяти предложениям.

In [34]:
df[['ru', 'clean_tokens']].head()

,ru,clean_tokens
0,"Да, но не совсем...",[]
1,!,[]
2,Приводимое ниже расписание является предварите...,"[приводить, ниже, расписание, являться, предва..."
3,"Но сейчас ...я вверяю вам удостовериться, что ...","[вверять, удостовериться, шотландец, приуменьш..."
4,Они тусовались там несколько недель.,"[тусоваться, несколько, неделя]"
